# 1.17 从字典中提取子集

## 有构造的字典 他是另外一个字典的子集

1. 最简单使用 字典推导
2. 通过创建一个元组序列 后将之传至dict()func 实现

In [1]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
# Make a dictionary of all prices over 200
p1 = {key: value for key, value in prices.items() if value > 200}
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p2 = {key: value for key, value in prices.items() if key in tech_names}

In [2]:
print(p1,'\n',p2)

{'AAPL': 612.78, 'IBM': 205.55} 
 {'HPQ': 37.2, 'AAPL': 612.78, 'IBM': 205.55}


### 大多数情况下 字典推导能做到的，可通过创建一个元组sequence 然后将之传至 dict() func 也可

In [4]:
p3 = dict((key, value) for key, value in prices.items() if value > 200)
print(p3)

{'AAPL': 612.78, 'IBM': 205.55}


### but 字典推导表达意思更加清晰 同时 运行速度更快(近一倍)<br>同时 第二个例子程序 可重写

In [8]:
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p4 = {key: prices[key] for key in prices.keys() & tech_names}
print(p4)

{'HPQ': 37.2, 'AAPL': 612.78, 'IBM': 205.55}


In [10]:
# p4 = {key: prices[key] for key in prices.keys() and tech_names}

### 上述两行推导 -- '&' 与 'and' 区别：
1. 一个是位运算
    ``num1 & num2 等同于 bin(num1) & bin(num2)``
2. 一个是逻辑运算
    ``num1 and num2 <<if num1 id False => False, else => num2>>[and 与 只要有一个假即假]``
------------------------------------------
在这里 & 的作用是 匹配 prices.keys() 与 tech_names 中 所含元素 相匹配的
结合以下描述,虽然 prices.keys() tech_names 两者类型but

In [11]:
prices.keys()

dict_keys(['ACME', 'HPQ', 'AAPL', 'IBM', 'FB'])

In [12]:
tech_names

{'AAPL', 'HPQ', 'IBM', 'MSFT'}

In [16]:
type(prices.keys()) == type(tech_names)

False

In [17]:
prices.keys() & tech_names

{'AAPL', 'HPQ', 'IBM'}

In [18]:
type(prices.keys() & tech_names)

set